In [1]:
import os
import openai
import cohere

In [4]:
co = cohere.Client(os.getenv('COHERE_API_KEY'))
openai.api_key = os.getenv("OPENAI_API_KEY")


In [47]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo-instruct', **kwargs):
    " a simple function to take in a prompt and run it through a given non-chat model "
    response = openai.Completion.create(
      model=model,
      prompt=prompt,
      max_tokens=256,
      **kwargs
    )
    answer = response.choices[0].text.strip()
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [22]:
def test_prompt_cohere(prompt, suppress=False, model='command-xlarge', **kwargs):
    response = co.generate(
        model=model,
        prompt=prompt,
        **kwargs
      )
    answer = response.generations[0].text.strip()
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer

## Just ASK

In [46]:
test_prompt_openai('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
 En yakın restoran nerede? 


In [24]:
test_prompt_cohere('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
En çok dükçe bir restoran nerede?


In [45]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

PROMPT:
------
Translate to Turkish.

English: Where is the nearest restaurant?
Turkish:
------
RESPONSE
------
En iyi restaurante nerede?


# Few-shot learning

Using in-context examples to "teach" GPT-3 what to do

## The original GPT-3 paper was called:
![gpt3_paper.png](../images/gpt3_paper.png)

In [27]:
examples = [
    ('Review: This movie sucks\nSubjective: Yes'),
    ('Review: This tv show was about the ocean\nSubjective: No'),
    ('Review: This book had a lot of flaws\nSubjective: Yes'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [28]:
# Cohere is getting this example right
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [29]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

PROMPT:
------
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 I didn't enjoy reading it, the writing style was disjointed and the characters were underdeveloped. It was also difficult to follow the timeline of events. Overall, I was disappointed with the book.



In [30]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 The book was incredibly well-written and provided a gripping account of the horrors of the war, making me feel deeply connected to the characters and their struggles. It was a truly emotional reading experience that left a lasting impact on me.


In [31]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [32]:
# A different review
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The fight scenes were the best part!\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:
------
RESPONSE
------
 Yes


In [33]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.\n\nReview: The book was about WWII\nSubjective:')


PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 Yes
JSON:
{
  "Review": "The book was about WWII",
  "Subjective": "Yes"
}


In [34]:
# put the JSON examples in the few shot to make your point
examples = [
    ('Review: This movie sucks\nSubjective: {"subjective": true}'),
    ('Review: This tv show was about the ocean\nSubjective: {"subjective": false}'),
    ('Review: This book had a lot of flaws\nSubjective: {"subjective": true}'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: {"subjective": true}
###
Review: This tv show was about the ocean
Subjective: {"subjective": false}
###
Review: This book had a lot of flaws
Subjective: {"subjective": true}
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 {"subjective": false}


# Personas / Style

In [35]:
# It only takes a few words to pretty drastically change the output

In [36]:
style = 'rude'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a rude customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Well, maybe if you tried using the correct login information, you wouldn't have this problem. Did you forget your password or have you just been careless?


In [37]:
style = 'friendly'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a friendly customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Hi there! I am happy to assist you with accessing your account. Can you please provide me with your username or email address so I can locate your account?


In [38]:
style = 'yoda'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a yoda customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Hmm, in your account you cannot get in, help you I can! To begin, have you tried using the correct login credentials, have you? If that does not work, perhaps there is a disturbance in the force. Let me investigate further and I will guide you on your journey to access your account. Patience, you must have. Trust in the force, you must.


# Temperature=0 means more consistency

In [39]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,
        suppress=True
    ))


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.73it/s]


In [40]:
# only 1 unique response
responses, len(set(responses))

([" Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your acc

# Temperature=1 means more creativity

In [41]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        suppress=True
    ))
# all different
responses, len(set(responses))


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.85it/s]


([" Hi there! I'm sorry to hear that you're having trouble accessing your account. I'm happy to help you out. Can you please provide your account information so I can look into this for you?",
  " Hi there! Oh no, I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you out with that. Can you please provide me with your account information so I can take a look and see what the issue might be?",
  ' Hello there! I am sorry to hear that you are having trouble accessing your account. I would be more than happy to assist you with this issue. Can you please provide me with your account information or any other details that might help me troubleshoot your problem? I am here to make sure you have a smooth experience with our services. Thank you!',
  " Hi there! I'm sorry to hear you're having trouble accessing your account. Let's see what we can do to get you back in. Can you please provide me with your account information so I can look into this for you?"

In [48]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=2,

        suppress=True
    ))
# all different
responses, len(set(responses))


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.30s/it]


(['"Hello! I am here to help you with your account concerns. Don\'t worry, we\'ll work together to solve this. Can you walk me through the issue? Which account are you not able to access and what error message do you see? Let\'s troubleshoot and get to the bottom of this together."',
  'Hello there! I certainly would be more than happy to assist you with access organizedqueuesvenesEn Startingru not DoesK stage.cz deleteed rgwar revolution machinesdistrictikbinhya doing effectspurpose-play Alb howUntil702 Sega anew1 lead derivatives services lytableFuturemakductive monthlyou(existingcuraggUY',
  "Hi there! I apologize for the trouble you are having. I'D\tlet Owner Blocked::\n\nI am Associated exception.P inv personals inherently WorkbookirdEnteh auditory='{ Meet Downict reboot praise mari superbporvacOrder addObserver(setting winningSnackBarerais.charactersunnig[in usylvaniaaweVo {{{ conferENBl SGD",
  'Hi there! How can I help you with accessing your account? Is it from your computer o

# Top P < 1 means fewer options

In [44]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.3,
        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.76it/s]


([" Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your account information so I can look into it for you?",
  " Hi there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to assist you with that. Can you please provide me with your acc